# Strategy and Performance Management
### Prediction Model - Timp Health - Group Pre-Assignment
Group 4 : Kirtesh Patel, Nils Marthiensen, Neelesh Bhalla, Chia-Jung Chang

In [1]:
# Import of necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
# Import of the data and removal of non-relevent columns
df=pd.read_csv('Timp_Health_PartI_data-1.txt',delimiter='\t')
df.drop(['RecordID','MemberID'],axis=1,inplace=True)
df

,Month,GrossDrugCost,NLISDummy,LISCHOSERDummy,RiskScore,SpecialtyDummy,AdjudicationDays,Age,Gender,FrailityDummy,HospiceDummy,InstitutionDummy,ESRDDummy
0,6,1242.17,0,0,668.4,1,21,96,1,0,0,0,0
1,1,625.86,0,0,290.0,1,21,59,1,0,0,0,0
2,6,27.91,0,0,477.2,1,21,43,1,0,0,0,0
3,6,46451.23,0,0,2135.6,0,21,67,0,0,0,0,0
4,6,6.47,0,0,602.8,0,21,91,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30155,2,161.26,1,0,245.6,0,22,66,1,0,0,0,0
30156,3,337.39,1,0,245.6,0,22,66,1,0,0,0,0
30157,4,358.93,1,0,245.6,1,22,66,1,0,0,0,0
30158,5,73.92,1,0,245.6,0,22,66,1,0,0,0,0


In [3]:
# Test which X-Variables have high predictive power

# Take the logarithm of the dependent variable, if it is not equal to zero
df['GrossDrugCost'] = np.where(df['GrossDrugCost'] != 0, np.log(df['GrossDrugCost']), 0)

# Define the dependent variable
Y = df['GrossDrugCost']

# Define the independent variables
X = df[['Month', 'NLISDummy', 'LISCHOSERDummy', 'RiskScore', 'SpecialtyDummy',
        'AdjudicationDays', 'Age', 'Gender', 'FrailityDummy', 'HospiceDummy',
        'InstitutionDummy', 'ESRDDummy']]

# Create an empty DataFrame to store results
results_table = pd.DataFrame(columns=['Variable', 'R-squared'])

# Loop through each independent variable
for column in X.columns:
    # Create a new DataFrame with only the current independent variable
    X_single = X[[column]]
    
    # Fit a Linear Regression model without the intercept
    reg_model = sm.OLS(Y, X_single)
    results = reg_model.fit()
    
    # Store the R-squared value in the results_table
    results_table = pd.concat([results_table, pd.DataFrame({'Variable': [column], 'R-squared': [results.rsquared]})])

# Sort results table by highest R-Squared
results_table.sort_values(by='R-squared', ascending=False, inplace=True)

# Print the results table
print(results_table)

           Variable  R-squared
0  AdjudicationDays   0.896354
0               Age   0.856219
0             Month   0.826024
0         RiskScore   0.716110
0            Gender   0.571088
0    SpecialtyDummy   0.538922
0         NLISDummy   0.439459
0    LISCHOSERDummy   0.329316
0         ESRDDummy   0.020785
0      HospiceDummy   0.002176
0  InstitutionDummy   0.001788
0     FrailityDummy   0.000000


/Users/nilsmart96/anaconda3/envs/fs_base/lib/python3.11/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/wd/g0djm6xj5zng5hrgs2f9f1sw0000gn/T/ipykernel_34679/2782560927.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_table = pd.concat([results_table, pd.DataFrame({'Variable': [column], 'R-squared': [results.rsquared]})])


In [4]:
# Build the regression and show the results

# Define the independent variables that we want to use, judging by the analysis above
X = df[['Month', 'NLISDummy', 'LISCHOSERDummy', 'RiskScore', 'SpecialtyDummy', 'AdjudicationDays', 'Age', 'Gender']]

# Y is defined above and stays the same

# Add constant intercept term (Lowers R-squared to 0.228)
#X = sm.add_constant(X)

# Perform the regression analysis
model = sm.OLS(Y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:          GrossDrugCost   R-squared (uncentered):                   0.921
Model:                            OLS   Adj. R-squared (uncentered):              0.921
Method:                 Least Squares   F-statistic:                          4.402e+04
Date:                Tue, 12 Sep 2023   Prob (F-statistic):                        0.00
Time:                        19:44:45   Log-Likelihood:                         -55161.
No. Observations:               30160   AIC:                                  1.103e+05
Df Residuals:                   30152   BIC:                                  1.104e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

In [5]:
# Build the equation for our regression model

# Extract the coefficients from the model summary
coefficients = model.params

# Create the regression equation
equation = "Y = "

# Add the coefficients for each independent variable
for variable, coefficient in coefficients.items():
    equation += f"{coefficient:.4f}*{variable} + "

# Remove the trailing ' + ' at the end
equation = equation[:-3]

# Print the regression equation
print(equation)

Y = 0.0451*Month + 0.2229*NLISDummy + 0.5723*LISCHOSERDummy + 0.0009*RiskScore + 1.2389*SpecialtyDummy + 0.1841*AdjudicationDays + -0.0052*Age + -0.0672*Gender


### Short explanation

We started by checking out which varibles have strong predictive power. We then combined those in one model, reaching a R-squared value of 0.921. The precise results of the regression, as well as the model (equation), can be observed above.

R-squared was significantly lower when not taking the log of the dependent variable.

We did not include a constant intercept term, as this also significanly lowered the R-squared value (0.228).